In [ ]:
import pandas as pd
import numpy as np
import glob
import os
import rasterio
from rasterio.windows import Window
from scipy.spatial import distance_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import (
    Input,
    Conv2D,
    MaxPooling2D,
    Flatten,
    Dense,
    Concatenate,
    Dropout,
    Layer,
    LayerNormalization,
    Lambda
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import Sequence
import tensorflow as tf
import gc # Import garbage collector
import pickle # For saving and loading the scaler and feature importance results
import json # For saving the feature importance results

# Define the single buffer size to use
BUFFER_METERS = 500

# ==================== 1. Load Data ==================== #
# NOTE: The data loading logic remains the same as it provides the inputs
# required for the new model architecture.
orig = pd.read_csv("../../data/RainySeason.csv")
river_100 = pd.read_csv("../data/Samples_100.csv")

# Define the columns to drop and the numeric columns to use for MLP
drop_cols = ['Stations','River','Lat','Long','geometry']
numeric_cols = orig.drop(columns=drop_cols).columns.drop('RI')

# Ensure there are no NaNs in the numeric columns before proceeding
orig[numeric_cols] = orig[numeric_cols].fillna(0)
river_100[numeric_cols] = river_100[numeric_cols].fillna(0)

# Train-test split
train_orig = orig.sample(10, random_state=42)
test_orig = orig.drop(train_orig.index)
train_combined = pd.concat([river_100, train_orig], ignore_index=True)

# ==================== 2. Collect ALL Rasters ==================== #
raster_paths = []
raster_paths += glob.glob("../CalIndices/*.tif")
raster_paths += glob.glob("../LULCMerged/*.tif")
raster_paths += glob.glob("../IDW/*.tif")

print(f"Using {len(raster_paths)} raster layers for CNN input.")
for r in raster_paths:
    print("   -", os.path.basename(r))

# ==================== 3. Create a Custom Data Generator ==================== #
def extract_patch_for_generator(coords, raster_files, buffer_pixels_x, buffer_pixels_y, patch_width, patch_height):
    """
    Extracts a batch of patches from rasters for a given set of coordinates.
    This function is optimized to be called by the data generator for each batch.
    
    This version includes robust NaN handling to prevent model training errors.
    """
    patches = []
    # Loop through each coordinate pair in the batch
    for lon, lat in coords:
        channels = []
        # Loop through each raster file to get a single patch for each raster
        for rfile in raster_files:
            with rasterio.open(rfile) as src:
                try:
                    row, col = src.index(lon, lat)
                    win = Window(col - buffer_pixels_x, row - buffer_pixels_y, patch_width, patch_height)
                    arr = src.read(1, window=win, boundless=True, fill_value=0)
                    
                    # Corrected logic: Convert any NaNs to a numerical value, e.g., 0,
                    # to prevent them from propagating through the model.
                    arr = np.nan_to_num(arr, nan=0.0)
                    
                    arr = arr.astype(np.float32)

                    # Get the maximum value, but check if it's a valid number and > 0.
                    # This prevents division by zero if the patch is all zeros.
                    max_val = np.nanmax(arr)
                    if np.isfinite(max_val) and max_val > 0:
                        arr /= max_val
                        
                except Exception as e:
                    print(f"Error processing {rfile} for coordinates ({lon}, {lat}): {e}")
                    arr = np.zeros((patch_width, patch_height), dtype=np.float32)
            channels.append(arr)
        patches.append(np.stack(channels, axis=-1))
    
    return np.array(patches)

class DataGenerator(Sequence):
    def __init__(self, coords, mlp_data, gnn_data, y, raster_paths, buffer_meters, batch_size=4, shuffle=True, **kwargs):
        super().__init__(**kwargs)
        self.coords = coords
        self.mlp_data = mlp_data
        self.gnn_data = gnn_data
        self.y = y
        self.raster_paths = raster_paths
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indices = np.arange(len(self.y))
        self.buffer_meters = buffer_meters

        # Pre-calculate patch size from the first raster
        with rasterio.open(raster_paths[0]) as src:
            res_x, res_y = src.res
            self.buffer_pixels_x = int(self.buffer_meters / res_x)
            self.buffer_pixels_y = int(self.buffer_meters / res_y)
            self.patch_width = 2 * self.buffer_pixels_x
            self.patch_height = 2 * self.buffer_pixels_y

        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.y) / self.batch_size))

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)
            
    def __getitem__(self, index):
        # Get batch indices
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]

        # Get batch data
        batch_coords = self.coords[batch_indices]
        batch_mlp = self.mlp_data[batch_indices]
        
        # Slice the GNN adjacency matrix for the current batch
        batch_gnn = self.gnn_data[batch_indices, :]

        batch_y = self.y[batch_indices]

        # Extract CNN patches for the current batch
        batch_cnn = extract_patch_for_generator(
            batch_coords,
            self.raster_paths,
            self.buffer_pixels_x,
            self.buffer_pixels_y,
            self.patch_width,
            self.patch_height
        )

        # Return a tuple of inputs and the target, which Keras expects
        return (batch_cnn, batch_mlp, batch_gnn), batch_y


# ==================== 4. Prepare GNN & MLP Input (only once) ==================== #
coords_train = train_combined[['Long','Lat']].values
coords_test = test_orig[['Long','Lat']].values

# We now split the training data into a training and validation set
train_split, val_split = train_test_split(train_combined, test_size=0.2, random_state=42)

coords_train_split = train_split[['Long','Lat']].values
coords_val_split = val_split[['Long','Lat']].values

dist_mat_train_split = distance_matrix(coords_train_split, coords_train_split)
gnn_train_split = np.exp(-dist_mat_train_split/10)
dist_mat_test_train = distance_matrix(coords_test, coords_train_split)
gnn_test = np.exp(-dist_mat_test_train/10)
dist_mat_val_train = distance_matrix(coords_val_split, coords_train_split)
gnn_val_split = np.exp(-dist_mat_val_train/10)


scaler = StandardScaler()
mlp_train_split = scaler.fit_transform(train_split[numeric_cols])
mlp_val_split = scaler.transform(val_split[numeric_cols])
mlp_test = scaler.transform(test_orig[numeric_cols])
y_train_split = train_split['RI'].values
y_val_split = val_split['RI'].values
y_test = test_orig['RI'].values

# ==================== 5. Define the Mixture of Experts Model ==================== #
def build_moe_model(patch_shape, gnn_dim, mlp_dim):
    # Inputs for all branches
    cnn_input = Input(shape=patch_shape, name="cnn_input")
    mlp_input = Input(shape=(mlp_dim,), name="mlp_input")
    gnn_input = Input(shape=(gnn_dim,), name="gnn_input")
    
    # --- Expert 1: CNN Branch ---
    cnn_branch = Conv2D(32, (3,3), activation="relu", padding="same")(cnn_input)
    cnn_branch = MaxPooling2D((2,2))(cnn_branch)
    cnn_branch = Conv2D(64, (3,3), activation="relu", padding="same")(cnn_branch)
    cnn_branch = MaxPooling2D((2,2))(cnn_branch)
    cnn_branch_flattened = Flatten()(cnn_branch)
    cnn_branch_dense = Dense(128, activation="relu")(cnn_branch_flattened)
    # The CNN expert's final prediction
    cnn_expert_out = Dense(1, activation="linear", name="cnn_expert_out")(cnn_branch_dense)

    # --- Expert 2: MLP Branch ---
    mlp_branch = Dense(64, activation="relu")(mlp_input)
    mlp_branch = Dense(32, activation="relu")(mlp_branch)
    # The MLP expert's final prediction
    mlp_expert_out = Dense(1, activation="linear", name="mlp_expert_out")(mlp_branch)

    # --- Expert 3: GNN Branch ---
    gnn_branch = Dense(64, activation="relu")(gnn_input)
    gnn_branch = Dense(32, activation="relu")(gnn_branch)
    # The GNN expert's final prediction
    gnn_expert_out = Dense(1, activation="linear", name="gnn_expert_out")(gnn_branch)

    # --- Gating Network ---
    # The gating network needs features from all inputs to make its decision.
    # We use the outputs of the dense layers before the final predictions as features.
    gate_input = Concatenate()([cnn_branch_dense, mlp_branch, gnn_branch])
    gate_network = Dense(64, activation="relu")(gate_input)
    gate_network = Dense(32, activation="relu")(gate_network)
    # The output is a set of weights for each expert (summing to 1 via softmax)
    gate_weights = Dense(3, activation="softmax", name="gate_weights")(gate_network)

    # --- Combine Experts and Gating Network ---
    # Stack the predictions from each expert.
    # The shape will be (batch_size, 3)
    experts_stack = Concatenate(axis=1, name="experts_stack")([cnn_expert_out, mlp_expert_out, gnn_expert_out])
    
    # Perform the weighted sum.
    # This is done using a Lambda layer which takes the experts' outputs and
    # the gating network's weights, and computes the dot product for each sample.
    final_output = Lambda(lambda x: tf.reduce_sum(x[0] * x[1], axis=1, keepdims=True), name="final_output")([experts_stack, gate_weights])

    # Build and compile the model
    model = Model(inputs=[cnn_input, mlp_input, gnn_input], outputs=final_output)
    model.compile(optimizer=Adam(learning_rate=0.0005), loss="mse")
    return model

def evaluate_model(model, coords_test, mlp_test, gnn_test_matrix, y_test, raster_paths, buffer_meters, batch_size=4, return_preds=False):
    num_samples = len(y_test)
    y_pred_list = []
    
    with rasterio.open(raster_paths[0]) as src:
        res_x, res_y = src.res
        buffer_pixels_x = int(buffer_meters / res_x)
        buffer_pixels_y = int(buffer_meters / res_y)
        patch_width = 2 * buffer_pixels_x
        patch_height = 2 * buffer_pixels_y

    for i in range(0, num_samples, batch_size):
        batch_coords = coords_test[i:i+batch_size]
        batch_mlp = mlp_test[i:i+batch_size]
        
        batch_gnn = gnn_test_matrix[i:i+batch_size, :]
        batch_y = y_test[i:i+batch_size]

        batch_cnn = extract_patch_for_generator(
            batch_coords,
            raster_paths,
            buffer_pixels_x,
            buffer_pixels_y,
            patch_width,
            patch_height
        )
        
        y_pred_list.append(model.predict((batch_cnn, batch_mlp, batch_gnn)).flatten())
        
    y_pred = np.concatenate(y_pred_list)
    
    if return_preds:
        return y_pred
    else:
        r2 = r2_score(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        return r2, rmse


# ==================== Run the Analysis ==================== #
print("\n" + "="*80)
print(f"Analyzing for BUFFER_METERS = {BUFFER_METERS}m")
print("="*80)

batch_size = 4
gnn_input_dim = len(train_split)

# Calculate CNN patch shape based on the current buffer size
with rasterio.open(raster_paths[0]) as src:
    res_x, res_y = src.res
    buffer_pixels_x = int(BUFFER_METERS / res_x)
    patch_width = 2 * buffer_pixels_x
    cnn_patch_shape = (patch_width, patch_width, len(raster_paths))

model = build_moe_model(cnn_patch_shape, gnn_input_dim, mlp_train_split.shape[1])
model.summary()

# ==================== 6. Create Data Generators ==================== #
train_generator = DataGenerator(
    coords=coords_train_split,
    mlp_data=mlp_train_split,
    gnn_data=gnn_train_split,
    y=y_train_split,
    raster_paths=raster_paths,
    buffer_meters=BUFFER_METERS,
    batch_size=batch_size,
    shuffle=True
)

validation_generator = DataGenerator(
    coords=coords_val_split,
    mlp_data=mlp_val_split,
    gnn_data=gnn_val_split,
    y=y_val_split,
    raster_paths=raster_paths,
    buffer_meters=BUFFER_METERS,
    batch_size=batch_size,
    shuffle=False # No need to shuffle validation data
)


# ==================== 7. Train Model ==================== #
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

history = model.fit(
    train_generator,
    epochs=100,
    verbose=1,
    callbacks=[early_stopping],
    validation_data=validation_generator
)

# ==================== 8. Evaluate ==================== #
y_pred_train = model.predict(train_generator).flatten()
r2_train = r2_score(y_train_split[:len(y_pred_train)], y_pred_train)
rmse_train = np.sqrt(mean_squared_error(y_train_split[:len(y_pred_train)], y_pred_train))

r2_test, rmse_test = evaluate_model(model, coords_test, mlp_test, gnn_test, y_test, raster_paths, buffer_meters=BUFFER_METERS, batch_size=batch_size)

print(f"\n Mixture of Experts Model Performance ({BUFFER_METERS}m):")
print(f"R² Train: {r2_train:.4f} | RMSE Train: {rmse_train:.4f}")
print(f"R² Test: {r2_test:.4f} | RMSE Test: {rmse_test:.4f}")

# ==================== 9. Feature Importance Analysis ==================== #
print("\n" + "-"*50)
print(f"Feature Importance Analysis for {BUFFER_METERS}m")
print("-"*50)

# --- 9.1 Combined Feature Importance (by Model Branch) ---
y_pred_baseline = evaluate_model(model, coords_test, mlp_test, gnn_test, y_test, raster_paths, buffer_meters=BUFFER_METERS, batch_size=batch_size, return_preds=True)
baseline_r2 = r2_score(y_test, y_pred_baseline)

print(f"\nBaseline Performance on Test Set: R² = {baseline_r2:.4f}")

# Ablate CNN branch
with rasterio.open(raster_paths[0]) as src:
    res_x, res_y = src.res
    buffer_pixels_x = int(BUFFER_METERS / res_x)
    buffer_pixels_y = int(BUFFER_METERS / res_y)
    patch_width = 2 * buffer_pixels_x
    patch_height = 2 * buffer_pixels_y

cnn_test_ablated = np.zeros_like(extract_patch_for_generator(
    coords_test, raster_paths, buffer_pixels_x, buffer_pixels_y, patch_width, patch_height
))
y_pred_cnn_ablated = model.predict((cnn_test_ablated, mlp_test, gnn_test)).flatten()
r2_cnn_ablated = r2_score(y_test, y_pred_cnn_ablated)
importance_cnn = baseline_r2 - r2_cnn_ablated

# Ablate MLP branch
mlp_test_ablated = np.zeros_like(mlp_test)
y_pred_mlp_ablated = model.predict((extract_patch_for_generator(
    coords_test, raster_paths, buffer_pixels_x, buffer_pixels_y, patch_width, patch_height
), mlp_test_ablated, gnn_test)).flatten()
r2_mlp_ablated = r2_score(y_test, y_pred_mlp_ablated)
importance_mlp = baseline_r2 - r2_mlp_ablated

# Ablate GNN branch
gnn_test_ablated = np.zeros_like(gnn_test)
y_pred_gnn_ablated = model.predict((extract_patch_for_generator(
    coords_test, raster_paths, buffer_pixels_x, buffer_pixels_y, patch_width, patch_height
), mlp_test, gnn_test_ablated)).flatten()
r2_gnn_ablated = r2_score(y_test, y_pred_gnn_ablated)
importance_gnn = baseline_r2 - r2_gnn_ablated

print("\n--- Combined Feature Importance (by Model Branch) ---")
print(f"CNN Branch Importance (R² drop): {importance_cnn:.4f}")
print(f"MLP Branch Importance (R² drop): {importance_mlp:.4f}")
print(f"GNN Branch Importance (R² drop): {importance_gnn:.4f}")

# --- 9.2 MLP Feature Importance (Permutation-based) ---
mlp_feature_importance = {}
for i, feature_name in enumerate(numeric_cols):
    mlp_test_shuffled = np.copy(mlp_test)
    np.random.shuffle(mlp_test_shuffled[:, i])
    
    y_pred_shuffled = model.predict((extract_patch_for_generator(
        coords_test, raster_paths, buffer_pixels_x, buffer_pixels_y, patch_width, patch_height
    ), mlp_test_shuffled, gnn_test)).flatten()
    r2_shuffled = r2_score(y_test, y_pred_shuffled)
    
    importance = baseline_r2 - r2_shuffled
    mlp_feature_importance[feature_name] = importance

print("\n--- MLP Feature Importance (Permutation-based) ---")
sorted_importance = sorted(mlp_feature_importance.items(), key=lambda item: item[1], reverse=True)
for feature, importance in sorted_importance:
    print(f"{feature:<20}: {importance:.4f}")
    
# Garbage collect to free up memory
del model, history, train_generator, validation_generator
gc.collect()

# ==================== 10. Save Model and Results ==================== #
print("\n" + "="*80)
print("Saving Model and Analysis Results...")
print("="*80)

# Rebuild and re-train the model to ensure it's in a savable state.
# This is a good practice to avoid issues with saving during a live session.
# First, let's get the necessary dimensions again.
batch_size = 4
gnn_input_dim = len(train_split)
with rasterio.open(raster_paths[0]) as src:
    res_x, res_y = src.res
    buffer_pixels_x = int(BUFFER_METERS / res_x)
    patch_width = 2 * buffer_pixels_x
    cnn_patch_shape = (patch_width, patch_width, len(raster_paths))

final_model = build_moe_model(cnn_patch_shape, gnn_input_dim, mlp_train_split.shape[1])
# Re-fit the model on the full training and validation data
final_model.fit(
    DataGenerator(coords=coords_train_split, mlp_data=mlp_train_split, gnn_data=gnn_train_split, y=y_train_split, raster_paths=raster_paths, buffer_meters=BUFFER_METERS, batch_size=batch_size),
    epochs=2,
    callbacks=[EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)],
    validation_data=DataGenerator(coords=coords_val_split, mlp_data=mlp_val_split, gnn_data=gnn_val_split, y=y_val_split, raster_paths=raster_paths, buffer_meters=BUFFER_METERS, batch_size=batch_size, shuffle=False)
)

output_dir = "mixture_of_experts"
os.makedirs(output_dir, exist_ok=True)

# 10.1 Save the trained Keras model
model_name = "mixture_of_experts_model"
final_model.save(f'{output_dir}/{model_name}.keras')
print(f"Saved trained model to {output_dir}/{model_name}.keras")

# 10.2 Save the StandardScaler object
# Using pickle.dump for serialization
with open(f'{output_dir}/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
print(f"Saved StandardScaler object to {output_dir}/scaler.pkl")

# 10.3 Combine and save all feature importance results into a single file
feature_importance_results = {
    "combined_branch_importance": {
        "CNN_Importance_R2_drop": importance_cnn,
        "MLP_Importance_R2_drop": importance_mlp,
        "GNN_Importance_R2_drop": importance_gnn
    },
    "mlp_permutation_importance": mlp_feature_importance
}

# Using pickle.dump for serialization
with open(f'{output_dir}/feature_importance.pkl', 'wb') as f:
    pickle.dump(feature_importance_results, f)
print(f"Saved all feature importance results to {output_dir}/feature_importance.pkl")

# 10.4 Save the preprocessed data arrays for future use
np.save(f'{output_dir}/coords_train.npy', coords_train)
np.save(f'{output_dir}/mlp_train.npy', mlp_train_split)
np.save(f'{output_dir}/gnn_train.npy', gnn_train_split)
np.save(f'{output_dir}/y_train.npy', y_train_split)
np.save(f'{output_dir}/coords_test.npy', coords_test)
np.save(f'{output_dir}/mlp_test.npy', mlp_test)
np.save(f'{output_dir}/gnn_test.npy', gnn_test)
np.save(f'{output_dir}/y_test.npy', y_test)
print("Saved preprocessed data arrays to .npy files")

print("\nAll requested artifacts have been saved successfully.")


# AlphaEarth Integration Enabled

This notebook has been enhanced with AlphaEarth satellite embeddings.

## Integration Options:
- **Option A**: Replace indices with AlphaEarth (64 bands)
- **Option B**: Add AlphaEarth to features (RECOMMENDED)
- **Option C**: PCA-reduced AlphaEarth (20 components)
- **Option D**: MLP enhancement only

Expected improvement: +0.5% to +0.8% in R²

In [ ]:
# ==================== ALPHAEARTH CONFIGURATION ====================
import pandas as pd
import numpy as np
import os

# Select which AlphaEarth option to use
ALPHA_EARTH_OPTION = 'B'  # Options: A, B (recommended), C, D
USE_ALPHA_EARTH = True

# Paths to AlphaEarth data files (created by 00_AlphaEarth_Data_Preparation.ipynb)
option_file = f'Option_{ALPHA_EARTH_OPTION}_RainyAE.csv'  # or WinterAE

# Load AlphaEarth data
if os.path.exists(option_file):
    ae_data = pd.read_csv(option_file)
    print(f'Loaded AlphaEarth Option {ALPHA_EARTH_OPTION}')
    print(f'Shape: {ae_data.shape}')
else:
    print(f'WARNING: {option_file} not found')
    print('Please run 00_AlphaEarth_Data_Preparation.ipynb first')
    USE_ALPHA_EARTH = False


In [2]:
import pandas as pd
import numpy as np
import glob
import os
import rasterio
from rasterio.windows import Window
from scipy.spatial import distance_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input,
    Conv2D,
    MaxPooling2D,
    Flatten,
    Dense,
    Concatenate,
    Dropout,
    Layer,
    Lambda
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import Sequence
import tensorflow as tf
import gc # Import garbage collector
import sys
from io import StringIO # To capture print output
import pickle # For saving dictionaries and other objects
# Define the single buffer size to use
BUFFER_METERS = 500
N_SPLITS = 5 # Number of folds for cross-validation
# ==================== 1. Load Data ==================== #
orig = pd.read_csv("../../data/RainySeason.csv")
river_100 = pd.read_csv("../data/Samples_100.csv")
# Define the columns to drop and the numeric columns to use for MLP
drop_cols = ['Stations','River','Lat','Long','geometry']
numeric_cols = orig.drop(columns=drop_cols).columns.drop('RI')
# Ensure there are no NaNs in the numeric columns before proceeding
orig[numeric_cols] = orig[numeric_cols].fillna(0)
river_100[numeric_cols] = river_100[numeric_cols].fillna(0)
# Train-test split
train_orig = orig.sample(10, random_state=42)
test_orig = orig.drop(train_orig.index)
train_combined = pd.concat([river_100, train_orig], ignore_index=True)
# ==================== 2. Collect ALL Rasters ==================== #
raster_paths = []
raster_paths += glob.glob("../CalIndices/*.tif")
raster_paths += glob.glob("../LULCMerged/*.tif")
raster_paths += glob.glob("../IDW/*.tif")
print(f"Using {len(raster_paths)} raster layers for CNN input.")
for r in raster_paths:
    print("  -", os.path.basename(r))
# ==================== 3. Define Metric Functions ==================== #
def smape(y_true, y_pred):
    """
    Calculates the Symmetric Mean Absolute Percentage Error (SMAPE).
    """
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    # Avoid division by zero by setting the result to 0 where denominator is 0
    return np.mean(np.where(denominator == 0, 0, numerator / denominator)) * 100
# ==================== 4. Create a Custom Data Generator ==================== #
def extract_patch_for_generator(coords, raster_files, buffer_pixels_x, buffer_pixels_y, patch_width, patch_height):
    """
    Extracts a batch of patches from rasters for a given set of coordinates.
    This function is optimized to be called by the data generator for each batch.
    """
    patches = []
    # Loop through each coordinate pair in the batch
    for lon, lat in coords:
        channels = []
        # Loop through each raster file to get a single patch for each raster
        for rfile in raster_files:
            with rasterio.open(rfile) as src:
                try:
                    row, col = src.index(lon, lat)
                    win = Window(col - buffer_pixels_x, row - buffer_pixels_y, patch_width, patch_height)
                    arr = src.read(1, window=win, boundless=True, fill_value=0)
                    
                    # Corrected logic: Convert any NaNs to a numerical value, e.g., 0,
                    # to prevent them from propagating through the model.
                    arr = np.nan_to_num(arr, nan=0.0)
                    arr = arr.astype(np.float32)
                    # Get the maximum value, but check if it's a valid number and > 0.
                    # This prevents division by zero if the patch is all zeros.
                    max_val = np.nanmax(arr)
                    if np.isfinite(max_val) and max_val > 0:
                        arr /= max_val
                        
                except Exception as e:
                    print(f"Error processing {rfile} for coordinates ({lon}, {lat}): {e}")
                    arr = np.zeros((patch_width, patch_height), dtype=np.float32)
            channels.append(arr)
        patches.append(np.stack(channels, axis=-1))
    
    return np.array(patches)
class DataGenerator(Sequence):
    def __init__(self, coords, mlp_data, gnn_data, y, raster_paths, buffer_meters, batch_size=4, shuffle=True, **kwargs):
        super().__init__(**kwargs)
        self.coords = coords
        self.mlp_data = mlp_data
        self.gnn_data = gnn_data
        self.y = y
        self.raster_paths = raster_paths
        self.buffer_meters = buffer_meters
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indices = np.arange(len(self.y))
        # Pre-calculate patch size from the first raster
        with rasterio.open(raster_paths[0]) as src:
            res_x, res_y = src.res
            self.buffer_pixels_x = int(self.buffer_meters / res_x)
            self.buffer_pixels_y = int(self.buffer_meters / res_y)
            self.patch_width = 2 * self.buffer_pixels_x
            self.patch_height = 2 * self.buffer_pixels_y
        self.on_epoch_end()
    def __len__(self):
        return int(np.floor(len(self.y) / self.batch_size))
    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)
            
    def __getitem__(self, index):
        # Get batch indices
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        # Get batch data
        batch_coords = self.coords[batch_indices]
        batch_mlp = self.mlp_data[batch_indices]
        batch_gnn = self.gnn_data[batch_indices, :]
        batch_y = self.y[batch_indices]
        # Extract CNN patches for the current batch
        batch_cnn = extract_patch_for_generator(
            batch_coords,
            self.raster_paths,
            self.buffer_pixels_x,
            self.buffer_pixels_y,
            self.patch_width,
            self.patch_height
        )
        # Return a tuple of inputs and the target, which Keras expects
        return (batch_cnn, batch_mlp, batch_gnn), batch_y
# ==================== 5. Define the Mixture of Experts Model ==================== #
def build_moe_model(patch_shape, gnn_dim, mlp_dim):
    # Inputs for all branches
    cnn_input = Input(shape=patch_shape, name="cnn_input")
    mlp_input = Input(shape=(mlp_dim,), name="mlp_input")
    gnn_input = Input(shape=(gnn_dim,), name="gnn_input")
    
    # --- Expert 1: CNN Branch ---
    cnn_branch = Conv2D(32, (3,3), activation="relu", padding="same")(cnn_input)
    cnn_branch = MaxPooling2D((2,2))(cnn_branch)
    cnn_branch = Conv2D(64, (3,3), activation="relu", padding="same")(cnn_branch)
    cnn_branch = MaxPooling2D((2,2))(cnn_branch)
    cnn_branch_flattened = Flatten()(cnn_branch)
    cnn_branch_dense = Dense(128, activation="relu")(cnn_branch_flattened)
    cnn_expert_out = Dense(1, activation="linear", name="cnn_expert_out")(cnn_branch_dense)
    # --- Expert 2: MLP Branch ---
    mlp_branch = Dense(64, activation="relu")(mlp_input)
    mlp_branch = Dense(32, activation="relu")(mlp_branch)
    mlp_expert_out = Dense(1, activation="linear", name="mlp_expert_out")(mlp_branch)
    # --- Expert 3: GNN Branch ---
    gnn_branch = Dense(64, activation="relu")(gnn_input)
    gnn_branch = Dense(32, activation="relu")(gnn_branch)
    gnn_expert_out = Dense(1, activation="linear", name="gnn_expert_out")(gnn_branch)
    # --- Gating Network ---
    gate_input = Concatenate()([cnn_branch_dense, mlp_branch, gnn_branch])
    gate_network = Dense(64, activation="relu")(gate_input)
    gate_network = Dense(32, activation="relu")(gate_network)
    gate_weights = Dense(3, activation="softmax", name="gate_weights")(gate_network)
    # --- Combine Experts and Gating Network ---
    experts_stack = Concatenate(axis=1, name="experts_stack")([cnn_expert_out, mlp_expert_out, gnn_expert_out])
    final_output = Lambda(lambda x: tf.reduce_sum(x[0] * x[1], axis=1, keepdims=True), name="final_output")([experts_stack, gate_weights])
    # Build and compile the model
    model = Model(inputs=[cnn_input, mlp_input, gnn_input], outputs=final_output)
    model.compile(optimizer=Adam(learning_rate=0.0005), loss="mse")
    return model
def evaluate_model(model, coords, mlp_data, gnn_data, y, raster_paths, buffer_meters, batch_size=4):
    """
    Evaluates a model on the given data and returns a dictionary of metrics.
    """
    num_samples = len(y)
    y_pred_list = []
    
    # Get patch dimensions
    with rasterio.open(raster_paths[0]) as src:
        res_x, res_y = src.res
        buffer_pixels_x = int(buffer_meters / res_x)
        buffer_pixels_y = int(buffer_meters / res_y)
        patch_width = 2 * buffer_pixels_x
        patch_height = 2 * buffer_pixels_y
    for i in range(0, num_samples, batch_size):
        batch_coords = coords[i:i+batch_size]
        batch_mlp = mlp_data[i:i+batch_size]
        batch_gnn = gnn_data[i:i+batch_size, :]
        
        batch_cnn = extract_patch_for_generator(
            batch_coords,
            raster_paths,
            buffer_pixels_x,
            buffer_pixels_y,
            patch_width,
            patch_height
        )
        
        y_pred_list.append(model.predict((batch_cnn, batch_mlp, batch_gnn)).flatten())
        
    y_pred = np.concatenate(y_pred_list)
    
    r2 = r2_score(y, y_pred)
    mae = mean_absolute_error(y, y_pred)
    rmse = np.sqrt(mean_squared_error(y, y_pred))
    smape_val = smape(y, y_pred)
    return {
        'R2': r2,
        'MAE': mae,
        'RMSE': rmse,
        'SMAPE': smape_val
    }
# ==================== 6. Run K-Fold Cross-Validation ==================== #
print("="*80)
print(f"Starting {N_SPLITS}-Fold Cross-Validation...")
print("="*80)
# Create a folder to save models
model_save_dir = "models/mixture_of_experts"
os.makedirs(model_save_dir, exist_ok=True)
print(f"Models will be saved in: '{model_save_dir}'")
# Prepare data for K-Fold
combined_indices = np.arange(len(train_combined))
coords_all = train_combined[['Long','Lat']].values
data_all = train_combined[numeric_cols].values
gnn_input_all = np.exp(-distance_matrix(coords_all, coords_all)/10)
y_all = train_combined['RI'].values
batch_size = 4
# Store metrics for each fold
fold_metrics = []
test_metrics = []
# Initialize K-Fold
kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=42)
for fold, (train_indices, val_indices) in enumerate(kf.split(combined_indices)):
    print(f"\n--- Fold {fold+1}/{N_SPLITS} ---")
    # Split data for the current fold
    coords_train_fold = coords_all[train_indices]
    coords_val_fold = coords_all[val_indices]
    mlp_train_fold = data_all[train_indices]
    mlp_val_fold = data_all[val_indices]
    y_train_fold = y_all[train_indices]
    y_val_fold = y_all[val_indices]
    # Scale MLP data and prepare GNN matrices for the current fold
    scaler_fold = StandardScaler()
    mlp_train_scaled = scaler_fold.fit_transform(mlp_train_fold)
    mlp_val_scaled = scaler_fold.transform(mlp_val_fold)
    
    dist_mat_train_fold = distance_matrix(coords_train_fold, coords_train_fold)
    gnn_train_fold = np.exp(-dist_mat_train_fold/10)
    
    dist_mat_val_fold = distance_matrix(coords_val_fold, coords_train_fold)
    gnn_val_fold = np.exp(-dist_mat_val_fold/10)
    # Re-initialize and compile the model for each fold
    with rasterio.open(raster_paths[0]) as src:
        res_x, res_y = src.res
        buffer_pixels_x = int(BUFFER_METERS / res_x)
        patch_width = 2 * buffer_pixels_x
        cnn_patch_shape = (patch_width, patch_width, len(raster_paths))
    
    model = build_moe_model(cnn_patch_shape, len(coords_train_fold), mlp_train_fold.shape[1])
    
    # Create Data Generators for the current fold
    train_generator = DataGenerator(
        coords=coords_train_fold,
        mlp_data=mlp_train_scaled,
        gnn_data=gnn_train_fold,
        y=y_train_fold,
        raster_paths=raster_paths,
        buffer_meters=BUFFER_METERS,
        batch_size=batch_size,
        shuffle=True
    )
    
    val_generator = DataGenerator(
        coords=coords_val_fold,
        mlp_data=mlp_val_scaled,
        gnn_data=gnn_val_fold,
        y=y_val_fold,
        raster_paths=raster_paths,
        buffer_meters=BUFFER_METERS,
        batch_size=batch_size,
        shuffle=False
    )
    
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    )
    print("Starting model training...")
    history = model.fit(
        train_generator,
        epochs=100,
        verbose=1,
        callbacks=[early_stopping],
        validation_data=val_generator
    )
    print("Training complete.")
    # Evaluate on the validation set
    fold_result = evaluate_model(model, coords_val_fold, mlp_val_scaled, gnn_val_fold, y_val_fold, raster_paths, BUFFER_METERS, batch_size)
    fold_metrics.append(fold_result)
    print("Validation Metrics:")
    print(f"R²: {fold_result['R2']:.4f}")
    print(f"MAE: {fold_result['MAE']:.4f}")
    print(f"RMSE: {fold_result['RMSE']:.4f}")
    print(f"SMAPE: {fold_result['SMAPE']:.4f}%")
    # Prepare and evaluate on the independent test set
    dist_mat_test_train = distance_matrix(test_orig[['Long','Lat']].values, coords_train_fold)
    gnn_test_fold = np.exp(-dist_mat_test_train/10)
    
    mlp_test_scaled = scaler_fold.transform(test_orig[numeric_cols].values)
    test_result = evaluate_model(model, test_orig[['Long','Lat']].values, mlp_test_scaled, gnn_test_fold, test_orig['RI'].values, raster_paths, BUFFER_METERS, batch_size)
    test_metrics.append(test_result)
    print("\nIndependent Test Set Metrics:")
    print(f"R²: {test_result['R2']:.4f}")
    print(f"MAE: {test_result['MAE']:.4f}")
    print(f"RMSE: {test_result['RMSE']:.4f}")
    print(f"SMAPE: {test_result['SMAPE']:.4f}%")
    
    # Save the trained model for the current fold
    model.save(os.path.join(model_save_dir, f"fold_{fold+1}.keras"))
    print(f"Model for Fold {fold+1} saved.")
    # Clean up to free memory
    del model, train_generator, val_generator, early_stopping, history
    tf.keras.backend.clear_session()
    gc.collect()
# ==================== 7. Print Final Averages ==================== #
# Calculate average metrics
avg_fold_metrics = pd.DataFrame(fold_metrics).mean().to_dict()
avg_test_metrics = pd.DataFrame(test_metrics).mean().to_dict()
print("\n" + "="*80)
print("Final Average Metrics Across All Folds:")
print("="*80)
print("\nAverage Validation Metrics:")
print(f"Average R²: {avg_fold_metrics['R2']:.4f}")
print(f"Average MAE: {avg_fold_metrics['MAE']:.4f}")
print(f"Average RMSE: {avg_fold_metrics['RMSE']:.4f}")
print(f"Average SMAPE: {avg_fold_metrics['SMAPE']:.4f}%")
print("\nAverage Independent Test Set Metrics:")
print(f"Average R²: {avg_test_metrics['R2']:.4f}")
print(f"Average MAE: {avg_test_metrics['MAE']:.4f}")
print(f"Average RMSE: {avg_test_metrics['RMSE']:.4f}")
print(f"Average SMAPE: {avg_test_metrics['SMAPE']:.4f}%")


Using 26 raster layers for CNN input.
  - bui.tif
  - ndsi.tif
  - savi.tif
  - ndbsi.tif
  - ui.tif
  - ndwi.tif
  - ndbi.tif
  - awei.tif
  - evi.tif
  - mndwi.tif
  - ndvi.tif
  - LULC2020.tif
  - LULC2021.tif
  - LULC2022.tif
  - LULC2019.tif
  - LULC2018.tif
  - LULC2017.tif
  - Pb_R.tif
  - ClayR.tif
  - SandR.tif
  - CdR.tif
  - CrR.tif
  - AsR.tif
  - SiltR.tif
  - CuR.tif
  - NiR.tif
Starting 5-Fold Cross-Validation...
Models will be saved in: 'models/mixture_of_experts'

--- Fold 1/5 ---
Starting model training...
Epoch 1/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 8s 258ms/step - loss: 112657.0312 - val_loss: 54699.5117
Epoch 2/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 5s 239ms/step - loss: 39652.7578 - val_loss: 7466.8813
Epoch 3/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 5s 230ms/step - loss: 6815.3745 - val_loss: 6668.9634
Epoch 4/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 5s 245ms/step - loss: 6376.0435 - val_loss: 14088.3027
Epoch 5/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 5s 227ms/step - loss: 5425.4478 - val_loss: 12836.4707

In [2]:
import pandas as pd
import numpy as np
import glob
import os
import rasterio
from rasterio.windows import Window
from scipy.spatial import distance_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import (
    Input,
    Conv2D,
    MaxPooling2D,
    Flatten,
    Dense,
    Concatenate,
    Dropout,
    Layer,
    LayerNormalization,
    Lambda
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import Sequence
import tensorflow as tf
import gc # Import garbage collector
import pickle # For saving and loading the scaler and feature importance results
import json # For saving the feature importance results

# Define the single buffer size to use
BUFFER_METERS = 500

# ==================== New Metric Functions ==================== #

def smape(y_true, y_pred):
    """
    Symmetric Mean Absolute Percentage Error (SMAPE)
    A robust measure of accuracy.
    """
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    # Handle the case where both y_true and y_pred are zero to avoid division by zero
    return np.mean(np.divide(numerator, denominator, out=np.zeros_like(numerator), where=denominator != 0)) * 100

# ==================== 1. Load Data ==================== #
# NOTE: The data loading logic remains the same as it provides the inputs
# required for the new model architecture.
orig = pd.read_csv("../../data/RainySeason.csv")
river_100 = pd.read_csv("../data/Samples_100.csv")

# Define the columns to drop and the numeric columns to use for MLP
drop_cols = ['Stations','River','Lat','Long','geometry']
numeric_cols = orig.drop(columns=drop_cols).columns.drop('RI')

# Ensure there are no NaNs in the numeric columns before proceeding
orig[numeric_cols] = orig[numeric_cols].fillna(0)
river_100[numeric_cols] = river_100[numeric_cols].fillna(0)

# Train-test split
train_orig = orig.sample(10, random_state=42)
test_orig = orig.drop(train_orig.index)
train_combined = pd.concat([river_100, train_orig], ignore_index=True)

# ==================== 2. Collect ALL Rasters ==================== #
raster_paths = []
raster_paths += glob.glob("../CalIndices/*.tif")
raster_paths += glob.glob("../LULCMerged/*.tif")
raster_paths += glob.glob("../IDW/*.tif")

print(f"Using {len(raster_paths)} raster layers for CNN input.")
for r in raster_paths:
    print("   -", os.path.basename(r))

# ==================== 3. Create a Custom Data Generator ==================== #
def extract_patch_for_generator(coords, raster_files, buffer_pixels_x, buffer_pixels_y, patch_width, patch_height):
    """
    Extracts a batch of patches from rasters for a given set of coordinates.
    This function is optimized to be called by the data generator for each batch.
    
    This version includes robust NaN handling to prevent model training errors.
    """
    patches = []
    # Loop through each coordinate pair in the batch
    for lon, lat in coords:
        channels = []
        # Loop through each raster file to get a single patch for each raster
        for rfile in raster_files:
            with rasterio.open(rfile) as src:
                try:
                    row, col = src.index(lon, lat)
                    win = Window(col - buffer_pixels_x, row - buffer_pixels_y, patch_width, patch_height)
                    arr = src.read(1, window=win, boundless=True, fill_value=0)
                    
                    # Corrected logic: Convert any NaNs to a numerical value, e.g., 0,
                    # to prevent them from propagating through the model.
                    arr = np.nan_to_num(arr, nan=0.0)
                    
                    arr = arr.astype(np.float32)

                    # Get the maximum value, but check if it's a valid number and > 0.
                    # This prevents division by zero if the patch is all zeros.
                    max_val = np.nanmax(arr)
                    if np.isfinite(max_val) and max_val > 0:
                        arr /= max_val
                        
                except Exception as e:
                    print(f"Error processing {rfile} for coordinates ({lon}, {lat}): {e}")
                    arr = np.zeros((patch_width, patch_height), dtype=np.float32)
            channels.append(arr)
        patches.append(np.stack(channels, axis=-1))
    
    return np.array(patches)

class DataGenerator(Sequence):
    def __init__(self, coords, mlp_data, gnn_data, y, raster_paths, buffer_meters, batch_size=4, shuffle=True, **kwargs):
        super().__init__(**kwargs)
        self.coords = coords
        self.mlp_data = mlp_data
        self.gnn_data = gnn_data
        self.y = y
        self.raster_paths = raster_paths
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indices = np.arange(len(self.y))
        self.buffer_meters = buffer_meters

        # Pre-calculate patch size from the first raster
        with rasterio.open(raster_paths[0]) as src:
            res_x, res_y = src.res
            self.buffer_pixels_x = int(self.buffer_meters / res_x)
            self.buffer_pixels_y = int(self.buffer_meters / res_y)
            self.patch_width = 2 * self.buffer_pixels_x
            self.patch_height = 2 * self.buffer_pixels_y

        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.y) / self.batch_size))

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)
            
    def __getitem__(self, index):
        # Get batch indices
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]

        # Get batch data
        batch_coords = self.coords[batch_indices]
        batch_mlp = self.mlp_data[batch_indices]
        
        # Slice the GNN adjacency matrix for the current batch
        batch_gnn = self.gnn_data[batch_indices, :]

        batch_y = self.y[batch_indices]

        # Extract CNN patches for the current batch
        batch_cnn = extract_patch_for_generator(
            batch_coords,
            self.raster_paths,
            self.buffer_pixels_x,
            self.buffer_pixels_y,
            self.patch_width,
            self.patch_height
        )

        # Return a tuple of inputs and the target, which Keras expects
        return (batch_cnn, batch_mlp, batch_gnn), batch_y


# ==================== 4. Prepare GNN & MLP Input (only once) ==================== #
coords_train = train_combined[['Long','Lat']].values
coords_test = test_orig[['Long','Lat']].values

# We now split the training data into a training and validation set
train_split, val_split = train_test_split(train_combined, test_size=0.2, random_state=42)

coords_train_split = train_split[['Long','Lat']].values
coords_val_split = val_split[['Long','Lat']].values

dist_mat_train_split = distance_matrix(coords_train_split, coords_train_split)
gnn_train_split = np.exp(-dist_mat_train_split/10)
dist_mat_test_train = distance_matrix(coords_test, coords_train_split)
gnn_test = np.exp(-dist_mat_test_train/10)
dist_mat_val_train = distance_matrix(coords_val_split, coords_train_split)
gnn_val_split = np.exp(-dist_mat_val_train/10)


scaler = StandardScaler()
mlp_train_split = scaler.fit_transform(train_split[numeric_cols])
mlp_val_split = scaler.transform(val_split[numeric_cols])
mlp_test = scaler.transform(test_orig[numeric_cols])
y_train_split = train_split['RI'].values
y_val_split = val_split['RI'].values
y_test = test_orig['RI'].values

# ==================== 5. Define the Mixture of Experts Model ==================== #
def build_moe_model(patch_shape, gnn_dim, mlp_dim):
    # Inputs for all branches
    cnn_input = Input(shape=patch_shape, name="cnn_input")
    mlp_input = Input(shape=(mlp_dim,), name="mlp_input")
    gnn_input = Input(shape=(gnn_dim,), name="gnn_input")
    
    # --- Expert 1: CNN Branch ---
    cnn_branch = Conv2D(32, (3,3), activation="relu", padding="same")(cnn_input)
    cnn_branch = MaxPooling2D((2,2))(cnn_branch)
    cnn_branch = Conv2D(64, (3,3), activation="relu", padding="same")(cnn_branch)
    cnn_branch = MaxPooling2D((2,2))(cnn_branch)
    cnn_branch_flattened = Flatten()(cnn_branch)
    cnn_branch_dense = Dense(128, activation="relu")(cnn_branch_flattened)
    # The CNN expert's final prediction
    cnn_expert_out = Dense(1, activation="linear", name="cnn_expert_out")(cnn_branch_dense)

    # --- Expert 2: MLP Branch ---
    mlp_branch = Dense(64, activation="relu")(mlp_input)
    mlp_branch = Dense(32, activation="relu")(mlp_branch)
    # The MLP expert's final prediction
    mlp_expert_out = Dense(1, activation="linear", name="mlp_expert_out")(mlp_branch)

    # --- Expert 3: GNN Branch ---
    gnn_branch = Dense(64, activation="relu")(gnn_input)
    gnn_branch = Dense(32, activation="relu")(gnn_branch)
    # The GNN expert's final prediction
    gnn_expert_out = Dense(1, activation="linear", name="gnn_expert_out")(gnn_branch)

    # --- Gating Network ---
    # The gating network needs features from all inputs to make its decision.
    # We use the outputs of the dense layers before the final predictions as features.
    gate_input = Concatenate()([cnn_branch_dense, mlp_branch, gnn_branch])
    gate_network = Dense(64, activation="relu")(gate_input)
    gate_network = Dense(32, activation="relu")(gate_network)
    # The output is a set of weights for each expert (summing to 1 via softmax)
    gate_weights = Dense(3, activation="softmax", name="gate_weights")(gate_network)

    # --- Combine Experts and Gating Network ---
    # Stack the predictions from each expert.
    # The shape will be (batch_size, 3)
    experts_stack = Concatenate(axis=1, name="experts_stack")([cnn_expert_out, mlp_expert_out, gnn_expert_out])
    
    # Perform the weighted sum.
    # This is done using a Lambda layer which takes the experts' outputs and
    # the gating network's weights, and computes the dot product for each sample.
    final_output = Lambda(lambda x: tf.reduce_sum(x[0] * x[1], axis=1, keepdims=True), name="final_output")([experts_stack, gate_weights])

    # Build and compile the model
    model = Model(inputs=[cnn_input, mlp_input, gnn_input], outputs=final_output)
    model.compile(optimizer=Adam(learning_rate=0.0005), loss="mse")
    return model

def evaluate_model(model, coords_test, mlp_test, gnn_test_matrix, y_test, raster_paths, buffer_meters, batch_size=4, return_preds=False):
    num_samples = len(y_test)
    y_pred_list = []
    
    with rasterio.open(raster_paths[0]) as src:
        res_x, res_y = src.res
        buffer_pixels_x = int(buffer_meters / res_x)
        buffer_pixels_y = int(buffer_meters / res_y)
        patch_width = 2 * buffer_pixels_x
        patch_height = 2 * buffer_pixels_y

    for i in range(0, num_samples, batch_size):
        batch_coords = coords_test[i:i+batch_size]
        batch_mlp = mlp_test[i:i+batch_size]
        
        batch_gnn = gnn_test_matrix[i:i+batch_size, :]
        batch_y = y_test[i:i+batch_size]

        batch_cnn = extract_patch_for_generator(
            batch_coords,
            raster_paths,
            buffer_pixels_x,
            buffer_pixels_y,
            patch_width,
            patch_height
        )
        
        y_pred_list.append(model.predict((batch_cnn, batch_mlp, batch_gnn)).flatten())
        
    y_pred = np.concatenate(y_pred_list)
    
    if return_preds:
        return y_pred
    else:
        r2 = r2_score(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mae = mean_absolute_error(y_test, y_pred)
        smape_val = smape(y_test, y_pred)
        return r2, rmse, mae, smape_val


# ==================== Run the Analysis ==================== #
print("\n" + "="*80)
print(f"Analyzing for BUFFER_METERS = {BUFFER_METERS}m")
print("="*80)

batch_size = 4
gnn_input_dim = len(train_split)

# Calculate CNN patch shape based on the current buffer size
with rasterio.open(raster_paths[0]) as src:
    res_x, res_y = src.res
    buffer_pixels_x = int(BUFFER_METERS / res_x)
    patch_width = 2 * buffer_pixels_x
    cnn_patch_shape = (patch_width, patch_width, len(raster_paths))

model = build_moe_model(cnn_patch_shape, gnn_input_dim, mlp_train_split.shape[1])
model.summary()

# ==================== 6. Create Data Generators ==================== #
train_generator = DataGenerator(
    coords=coords_train_split,
    mlp_data=mlp_train_split,
    gnn_data=gnn_train_split,
    y=y_train_split,
    raster_paths=raster_paths,
    buffer_meters=BUFFER_METERS,
    batch_size=batch_size,
    shuffle=True
)

validation_generator = DataGenerator(
    coords=coords_val_split,
    mlp_data=mlp_val_split,
    gnn_data=gnn_val_split,
    y=y_val_split,
    raster_paths=raster_paths,
    buffer_meters=BUFFER_METERS,
    batch_size=batch_size,
    shuffle=False # No need to shuffle validation data
)


# ==================== 7. Train Model ==================== #
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=15,
    restore_best_weights=True
)

history = model.fit(
    train_generator,
    epochs=100,
    verbose=1,
    callbacks=[early_stopping],
    validation_data=validation_generator
)

# ==================== 8. Evaluate ==================== #
y_pred_train = model.predict(train_generator).flatten()
r2_train = r2_score(y_train_split[:len(y_pred_train)], y_pred_train)
rmse_train = np.sqrt(mean_squared_error(y_train_split[:len(y_pred_train)], y_pred_train))
mae_train = mean_absolute_error(y_train_split[:len(y_pred_train)], y_pred_train)
smape_train = smape(y_train_split[:len(y_pred_train)], y_pred_train)

r2_test, rmse_test, mae_test, smape_test = evaluate_model(model, coords_test, mlp_test, gnn_test, y_test, raster_paths, buffer_meters=BUFFER_METERS, batch_size=batch_size)

print(f"\n Mixture of Experts Model Performance ({BUFFER_METERS}m):")
print(f"R² Train: {r2_train:.4f} | RMSE Train: {rmse_train:.4f} | MAE Train: {mae_train:.4f} | SMAPE Train: {smape_train:.4f}")
print(f"R² Test: {r2_test:.4f} | RMSE Test: {rmse_test:.4f} | MAE Test: {mae_test:.4f} | SMAPE Test: {smape_test:.4f}")

# ==================== 9. Feature Importance Analysis ==================== #
print("\n" + "-"*50)
print(f"Feature Importance Analysis for {BUFFER_METERS}m")
print("-"*50)

# --- 9.1 Combined Feature Importance (by Model Branch) ---
y_pred_baseline = evaluate_model(model, coords_test, mlp_test, gnn_test, y_test, raster_paths, buffer_meters=BUFFER_METERS, batch_size=batch_size, return_preds=True)
baseline_r2 = r2_score(y_test, y_pred_baseline)
baseline_rmse = np.sqrt(mean_squared_error(y_test, y_pred_baseline))
baseline_mae = mean_absolute_error(y_test, y_pred_baseline)
baseline_smape = smape(y_test, y_pred_baseline)

print(f"\nBaseline Performance on Test Set: R² = {baseline_r2:.4f}, RMSE = {baseline_rmse:.4f}, MAE = {baseline_mae:.4f}, SMAPE = {baseline_smape:.4f}")

# Ablate CNN branch
with rasterio.open(raster_paths[0]) as src:
    res_x, res_y = src.res
    buffer_pixels_x = int(BUFFER_METERS / res_x)
    buffer_pixels_y = int(BUFFER_METERS / res_y)
    patch_width = 2 * buffer_pixels_x
    patch_height = 2 * buffer_pixels_y

cnn_test_ablated = np.zeros_like(extract_patch_for_generator(
    coords_test, raster_paths, buffer_pixels_x, buffer_pixels_y, patch_width, patch_height
))
y_pred_cnn_ablated = model.predict((cnn_test_ablated, mlp_test, gnn_test)).flatten()
r2_cnn_ablated = r2_score(y_test, y_pred_cnn_ablated)
rmse_cnn_ablated = np.sqrt(mean_squared_error(y_test, y_pred_cnn_ablated))
mae_cnn_ablated = mean_absolute_error(y_test, y_pred_cnn_ablated)
smape_cnn_ablated = smape(y_test, y_pred_cnn_ablated)
importance_cnn_r2 = baseline_r2 - r2_cnn_ablated
importance_cnn_rmse = rmse_cnn_ablated - baseline_rmse
importance_cnn_mae = mae_cnn_ablated - baseline_mae
importance_cnn_smape = smape_cnn_ablated - baseline_smape


# Ablate MLP branch
mlp_test_ablated = np.zeros_like(mlp_test)
y_pred_mlp_ablated = model.predict((extract_patch_for_generator(
    coords_test, raster_paths, buffer_pixels_x, buffer_pixels_y, patch_width, patch_height
), mlp_test_ablated, gnn_test)).flatten()
r2_mlp_ablated = r2_score(y_test, y_pred_mlp_ablated)
rmse_mlp_ablated = np.sqrt(mean_squared_error(y_test, y_pred_mlp_ablated))
mae_mlp_ablated = mean_absolute_error(y_test, y_pred_mlp_ablated)
smape_mlp_ablated = smape(y_test, y_pred_mlp_ablated)
importance_mlp_r2 = baseline_r2 - r2_mlp_ablated
importance_mlp_rmse = rmse_mlp_ablated - baseline_rmse
importance_mlp_mae = mae_mlp_ablated - baseline_mae
importance_mlp_smape = smape_mlp_ablated - baseline_smape

# Ablate GNN branch
gnn_test_ablated = np.zeros_like(gnn_test)
y_pred_gnn_ablated = model.predict((extract_patch_for_generator(
    coords_test, raster_paths, buffer_pixels_x, buffer_pixels_y, patch_width, patch_height
), mlp_test, gnn_test_ablated)).flatten()
r2_gnn_ablated = r2_score(y_test, y_pred_gnn_ablated)
rmse_gnn_ablated = np.sqrt(mean_squared_error(y_test, y_pred_gnn_ablated))
mae_gnn_ablated = mean_absolute_error(y_test, y_pred_gnn_ablated)
smape_gnn_ablated = smape(y_test, y_pred_gnn_ablated)
importance_gnn_r2 = baseline_r2 - r2_gnn_ablated
importance_gnn_rmse = rmse_gnn_ablated - baseline_rmse
importance_gnn_mae = mae_gnn_ablated - baseline_mae
importance_gnn_smape = smape_gnn_ablated - baseline_smape

print("\n--- Combined Feature Importance (by Model Branch) ---")
print(f"CNN Branch Importance (R² drop): {importance_cnn_r2:.4f} | (RMSE increase): {importance_cnn_rmse:.4f} | (MAE increase): {importance_cnn_mae:.4f} | (SMAPE increase): {importance_cnn_smape:.4f}")
print(f"MLP Branch Importance (R² drop): {importance_mlp_r2:.4f} | (RMSE increase): {importance_mlp_rmse:.4f} | (MAE increase): {importance_mlp_mae:.4f} | (SMAPE increase): {importance_mlp_smape:.4f}")
print(f"GNN Branch Importance (R² drop): {importance_gnn_r2:.4f} | (RMSE increase): {importance_gnn_rmse:.4f} | (MAE increase): {importance_gnn_mae:.4f} | (SMAPE increase): {importance_gnn_smape:.4f}")

# --- 9.2 MLP Feature Importance (Permutation-based) ---
mlp_feature_importance = {}
for i, feature_name in enumerate(numeric_cols):
    mlp_test_shuffled = np.copy(mlp_test)
    np.random.shuffle(mlp_test_shuffled[:, i])
    
    y_pred_shuffled = model.predict((extract_patch_for_generator(
        coords_test, raster_paths, buffer_pixels_x, buffer_pixels_y, patch_width, patch_height
    ), mlp_test_shuffled, gnn_test)).flatten()
    r2_shuffled = r2_score(y_test, y_pred_shuffled)
    rmse_shuffled = np.sqrt(mean_squared_error(y_test, y_pred_shuffled))
    mae_shuffled = mean_absolute_error(y_test, y_pred_shuffled)
    smape_shuffled = smape(y_test, y_pred_shuffled)
    
    importance_r2 = baseline_r2 - r2_shuffled
    importance_rmse = rmse_shuffled - baseline_rmse
    importance_mae = mae_shuffled - baseline_mae
    importance_smape = smape_shuffled - baseline_smape

    mlp_feature_importance[feature_name] = {
        "r2_drop": importance_r2,
        "rmse_increase": importance_rmse,
        "mae_increase": importance_mae,
        "smape_increase": importance_smape
    }

print("\n--- MLP Feature Importance (Permutation-based) ---")
sorted_importance = sorted(mlp_feature_importance.items(), key=lambda item: item[1]['r2_drop'], reverse=True)
for feature, importance in sorted_importance:
    print(f"{feature:<20}: R² drop={importance['r2_drop']:.4f}, RMSE increase={importance['rmse_increase']:.4f}, MAE increase={importance['mae_increase']:.4f}, SMAPE increase={importance['smape_increase']:.4f}")
    

print("\n" + "="*80)
print("Saving Model and Analysis Results...")
print("="*80)

batch_size = 4
gnn_input_dim = len(train_split)
with rasterio.open(raster_paths[0]) as src:
    res_x, res_y = src.res
    buffer_pixels_x = int(BUFFER_METERS / res_x)
    patch_width = 2 * buffer_pixels_x
    cnn_patch_shape = (patch_width, patch_width, len(raster_paths))

final_model = build_moe_model(cnn_patch_shape, gnn_input_dim, mlp_train_split.shape[1])

final_model.fit(
    DataGenerator(coords=coords_train_split, mlp_data=mlp_train_split, gnn_data=gnn_train_split, y=y_train_split, raster_paths=raster_paths, buffer_meters=BUFFER_METERS, batch_size=batch_size),
    epochs=100,
    callbacks=[EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)],
    validation_data=DataGenerator(coords=coords_val_split, mlp_data=mlp_val_split, gnn_data=gnn_val_split, y=y_val_split, raster_paths=raster_paths, buffer_meters=BUFFER_METERS, batch_size=batch_size, shuffle=False)
)


feature_importance_results = {
    "combined_branch_importance": {
        "CNN_Importance": {
            "R2_drop": importance_cnn_r2,
            "RMSE_increase": importance_cnn_rmse,
            "MAE_increase": importance_cnn_mae,
            "SMAPE_increase": importance_cnn_smape
        },
        "MLP_Importance": {
            "R2_drop": importance_mlp_r2,
            "RMSE_increase": importance_mlp_rmse,
            "MAE_increase": importance_mlp_mae,
            "SMAPE_increase": importance_mlp_smape
        },
        "GNN_Importance": {
            "R2_drop": importance_gnn_r2,
            "RMSE_increase": importance_gnn_rmse,
            "MAE_increase": importance_gnn_mae,
            "SMAPE_increase": importance_gnn_smape
        }
    },
    "mlp_permutation_importance": mlp_feature_importance
}


Using 26 raster layers for CNN input.
   - bui.tif
   - ndsi.tif
   - savi.tif
   - ndbsi.tif
   - ui.tif
   - ndwi.tif
   - ndbi.tif
   - awei.tif
   - evi.tif
   - mndwi.tif
   - ndvi.tif
   - LULC2020.tif
   - LULC2021.tif
   - LULC2022.tif
   - LULC2019.tif
   - LULC2018.tif
   - LULC2017.tif
   - Pb_R.tif
   - ClayR.tif
   - SandR.tif
   - CdR.tif
   - CrR.tif
   - AsR.tif
   - SiltR.tif
   - CuR.tif
   - NiR.tif

Analyzing for BUFFER_METERS = 500m


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ cnn_input           │ (None, 100, 100,  │          0 │ -                 │
│ (InputLayer)        │ 26)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 100, 100,  │      7,520 │ cnn_input[0][0]   │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_4     │ (None, 50, 50,    │          0 │ conv2d_4[0][0]    │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 50, 50,    │     18,496 │ max_pooling2d_4[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_5     │ (None, 25, 25,    │          0 │ conv2d_5[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlp_input           │ (None, 15)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gnn_input           │ (None, 88)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_2 (Flatten) │ (None, 40000)     │          0 │ max_pooling2d_5[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_15 (Dense)    │ (None, 64)        │      1,024 │ mlp_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_17 (Dense)    │ (None, 64)        │      5,696 │ gnn_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_14 (Dense)    │ (None, 128)       │  5,120,128 │ flatten_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_16 (Dense)    │ (None, 32)        │      2,080 │ dense_15[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_18 (Dense)    │ (None, 32)        │      2,080 │ dense_17[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 192)       │          0 │ dense_14[0][0],   │
│ (Concatenate)       │                   │            │ dense_16[0][0],   │
│                     │                   │            │ dense_18[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_19 (Dense)    │ (None, 64)        │     12,352 │ concatenate_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cnn_expert_out      │ (None, 1)         │        129 │ dense_14[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlp_expert_out      │ (None, 1)         │         33 │ dense_16[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gnn_expert_out      │ (None, 1)         │         33 │ dense_18[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_20 (Dense)    │ (None, 32)        │      2,080 │ dense_19[0][0]  

 Total params: 5,171,750 (19.73 MB)

 Trainable params: 5,171,750 (19.73 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 6s 181ms/step - loss: 123640.4297 - val_loss: 26903.3496
Epoch 2/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 181ms/step - loss: 24425.3301 - val_loss: 9253.4277
Epoch 3/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 181ms/step - loss: 7333.6240 - val_loss: 7654.9741
Epoch 4/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 169ms/step - loss: 6128.8706 - val_loss: 6512.0537
Epoch 5/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 173ms/step - loss: 3767.6982 - val_loss: 6391.6172
Epoch 6/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 182ms/step - loss: 5399.1704 - val_loss: 6149.3281
Epoch 7/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 172ms/step - loss: 4626.8452 - val_loss: 7612.5625
Epoch 8/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 168ms/step - loss: 5004.7456 - val_loss: 5740.0366
Epoch 9/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 201ms/step - loss: 4692.3413 - val_loss: 5518.9204
Epoch 10/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 189ms/step - loss: 4473.5977 - val_loss: 6472.9702
Epoch 11/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 190ms/step - loss: 326

In [4]:
feature_importance_results


{'combined_branch_importance': {'CNN_Importance': {'R2_drop': 5.519753933430636,
   'RMSE_increase': 163.21033107150362,
   'MAE_increase': 151.34430820465087,
   'SMAPE_increase': 162.6982412523215},
  'MLP_Importance': {'R2_drop': 0.0,
   'RMSE_increase': 0.0,
   'MAE_increase': 0.0,
   'SMAPE_increase': 0.0},
  'GNN_Importance': {'R2_drop': 0.0,
   'RMSE_increase': 0.0,
   'MAE_increase': 0.0,
   'SMAPE_increase': 0.0}},
 'mlp_permutation_importance': {'hydro_dist_brick': {'r2_drop': 0.0,
   'rmse_increase': 0.0,
   'mae_increase': 0.0,
   'smape_increase': 0.0},
  'num_brick_field': {'r2_drop': 0.0,
   'rmse_increase': 0.0,
   'mae_increase': 0.0,
   'smape_increase': 0.0},
  'hydro_dist_ind': {'r2_drop': 0.0,
   'rmse_increase': 0.0,
   'mae_increase': 0.0,
   'smape_increase': 0.0},
  'num_industry': {'r2_drop': 0.0,
   'rmse_increase': 0.0,
   'mae_increase': 0.0,
   'smape_increase': 0.0},
  'CrR': {'r2_drop': 0.0,
   'rmse_increase': 0.0,
   'mae_increase': 0.0,
   'smape_incr